In [ ]:
# Here I use the file 'no_immvar_fine_prop.csv' from the output of Fine_analysis_age-2024 to calculate cell proportion slopes

In [2]:
import numpy as np
import scipy.stats as stats
from statsmodels.stats.multitest import multipletests
from scipy.stats import linregress
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

import scanpy as sc
sc.settings.verbosity = 3
sc.logging.print_version_and_date()
import scanpy.external as sce

import statsmodels.api as sm
!pip list
from statsmodels.formula.api import ols


Running Scanpy 1.9.3, on 2024-05-23 15:41.
Package                            Version
---------------------------------- -------------------
alabaster                          0.7.12
anaconda-client                    1.7.2
anaconda-navigator                 2.0.3
anaconda-project                   0.9.1
anndata                            0.9.2
anyio                              2.2.0
appdirs                            1.4.4
argh                               0.26.2
argon2-cffi                        20.1.0
asn1crypto                         1.4.0
astroid                            2.5
astropy                            4.2.1
async-generator                    1.10
atomicwrites                       1.4.0
attrs                              20.3.0
autopep8                           1.5.6
Babel                              2.9.0
backcall                           0.2.0
backports.functools-lru-cache      1.6.4
backports.shutil-get-terminal-size 1.0.0
backports.tempfile                 1.0

In [25]:
# Figure settings
mpl.rcdefaults()
# Set font to be arial
# mpl.rc('font', **{'sans-serif':'Arial', 'size':12})
mpl.rc('font', **{'size':12})
mpl.rcParams['mathtext.rm'] = 'sans' # to have non-italic greek letter, use r'$\mathrm{\alpha}$', does NOT work with f-string
mpl.rcParams['axes.titlesize'] = 12
# Set default tick size
mpl.rcParams['xtick.major.size'] = 5.5
mpl.rcParams['ytick.major.size'] = 5.5
mpl.rcParams['xtick.minor.size'] = 2.5
mpl.rcParams['ytick.minor.size'] = 2.5
# Default legend settings
mpl.rcParams['legend.fancybox'] = False
mpl.rcParams['legend.edgecolor'] = 'k'

# sc.settings.set_figure_params(dpi=120)

In [27]:
## Determine p values for slope

## Inputs: data is a dataframe with cell type, proportion, corresponding sample, and corresponding sample metadata
## the number of rows is the number of cell types * the number of samples
## the number of columns is dependent on the amount of metadata
## y var is the y variable of interest (proportion)
## x var is the list of x variables part of the multiple linear regression model
## Outputs: the p value for the calculated slope between cell type proportion and age

def pval(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X = sm.add_constant(X)
    model = sm.OLS(Y, X)
    a = model.fit()
    p = a.pvalues.loc['age']
    return p

In [6]:
## Determine slope

## Inputs: data is a dataframe with cell type, proportion, corresponding sample, and corresponding sample metadata
## the number of rows is the number of cell types * the number of samples
## the number of columns is dependent on the amount of metadata
## y var is the y variable of interest (proportion)
## x var is the list of x variables part of the multiple linear regression model
## Outputs: the slope between cell type proportion and age

def slopey(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X = sm.add_constant(X)
    model = sm.OLS(Y, X)
    a = model.fit()
    s = a.params.loc['age']
    return s

In [15]:
## Complete calculations and calculate p and corrected p values

## Inputs: data is a dataframe with cell type, proportion, corresponding sample, and corresponding sample metadata
## the number of rows is the number of cell types * the number of samples
## the number of columns is dependent on the amount of metadata
## y var is the y variable of interest (proportion)
## x var is the set of x variables part of the multiple linear regression model (list)
## column is the column of interest (proportion)
## group is the name of the cell type column
## label is the cohort, either SLE or controls
## list of cell types is a list containing cell types
## Outputs: a dataframe with cell type as rownames, and the slope, p-val, and corrected p-val for each cell type
## for SLE and control groups

def nostd(data, yvar, xvars, column, group, label, list_of_cell_types):
    temp_slope = data.groupby(group).apply(slopey, yvar, xvars)
    temp_pval = data.groupby(group).apply(pval, yvar, xvars)
    corrected_pval = multipletests(temp_pval, method="fdr_bh")[1]
    df = pd.DataFrame(list(zip(temp_slope, temp_pval, corrected_pval)),
              columns=[('{} Slope').format(label), ('{} p-val').format(label), ('{} corrected p-val').format(label) ])
    df['author_cell_type'] = list_of_cell_types
    return df
    

In [3]:
## Read in data and separate by condition

fine_prop = pd.read_csv("no_immvar_fine_prop.csv")
fine_types = sorted(fine_prop["ct_cov"].unique())
LP_fine = fine_prop.loc[fine_prop['Group'] != 'Control']
control_fine = fine_prop.loc[fine_prop['Group'] == 'Control']

In [53]:
## Calculations for each condition

lupus_raw_data = nostd(LP_fine, 'prop', ['age', 'sex', 'raceeth'], 'prop', 'ct_cov', 'Lupus', fine_types)
control_raw_data = nostd(control_fine, 'prop', ['age', 'sex', 'raceeth'], 'prop', 'ct_cov', 'Control', fine_types)

In [43]:
fine_data_raw = lupus_raw_data.merge(control_raw_data, on = "author_cell_type", how = "outer")
fine_data_raw = fine_data_raw.set_index("author_cell_type")

In [44]:
fine_data_raw

,Lupus Slope,Lupus p-val,Lupus corrected p-val,Control Slope,Control p-val,Control corrected p-val
author_cell_type,,,,,,
B_atypical,0.000004,9.002718e-01,9.002718e-01,0.000059,0.122933,0.286844
B_mem,0.000067,5.899225e-01,8.130054e-01,0.000369,0.288459,0.448715
B_naive,0.000128,7.711473e-01,8.304664e-01,0.000069,0.818407,0.818407
B_plasma,-0.000016,8.937384e-02,2.502467e-01,-0.000003,0.796963,0.818407
CytoT_GZMH+,0.000232,6.069815e-01,8.130054e-01,0.000566,0.074239,0.207868
CytoT_GZMK+,0.000178,3.475723e-01,8.110020e-01,0.000257,0.284032,0.448715
NK_bright,-0.000034,5.849118e-01,8.130054e-01,-0.000042,0.326781,0.457494
NK_dim,0.000661,2.778825e-03,1.296785e-02,0.000348,0.377822,0.480865
Progen,-0.000298,1.663819e-13,2.329347e-12,-0.000351,0.000002,0.000027


In [ ]:
fine_data_raw.to_csv('fine_data_raw.csv', index=True)